## Part 1: Preprocessing

In [151]:
# Import our dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import matplotlib.pyplot as plt

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [152]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [153]:
# Print unique values in target variables
print("Unique Attrition values:", attrition_df['Attrition'].unique())
print("Unique Department values:", attrition_df['Department'].unique())

Unique Attrition values: ['Yes' 'No']
Unique Department values: ['Sales' 'Research & Development' 'Human Resources']


In [154]:
# Create x using selected columns
selected_columns = [
    'Age', 'BusinessTravel', 'DistanceFromHome', 'Education',
    'EducationField', 'EnvironmentSatisfaction', 'JobInvolvement',
    'JobLevel', 'JobSatisfaction', 'YearsSinceLastPromotion']
x = attrition_df[selected_columns].copy()

# Create separate DataFrames for features and targets
y_attrition = attrition_df['Attrition']
y_department = attrition_df['Department']

# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

# Display the new DataFrame
print(y_df.head())


  Attrition              Department
0       Yes                   Sales
1        No  Research & Development
2       Yes  Research & Development
3        No  Research & Development
4        No  Research & Development


In [155]:
# Preprocessing x features
categorical_cols = ['BusinessTravel', 'EducationField']
numerical_cols = [col for col in x.columns if col not in categorical_cols]

# Scale numerical features
scaler = StandardScaler()
x[numerical_cols] = scaler.fit_transform(x[numerical_cols])

# One-hot encode categorical features
x = pd.get_dummies(x, columns=categorical_cols, drop_first=True)

# Process target variables
# For Attrition (binary classification)
attrition_encoder = OneHotEncoder(sparse_output=False)
y_attrition = attrition_encoder.fit_transform(attrition_df['Attrition'].values.reshape(-1, 1))
print("Attrition encoded shape:", y_attrition.shape)
print("Attrition categories:", attrition_encoder.categories_)

# For Department (multi-class classification)
department_encoder = OneHotEncoder(sparse_output=False)
y_department = department_encoder.fit_transform(attrition_df['Department'].values.reshape(-1, 1))
print("Department encoded shape:", y_department.shape)
print("Department categories:", department_encoder.categories_)


Attrition encoded shape: (1470, 2)
Attrition categories: [array(['No', 'Yes'], dtype=object)]
Department encoded shape: (1470, 3)
Department categories: [array(['Human Resources', 'Research & Development', 'Sales'], dtype=object)]


In [156]:
# Split the data
x_train, x_test, y_dept_train, y_dept_test, y_attr_train, y_attr_test = train_test_split(
    x, y_attrition, y_department,
    test_size=0.2,
    random_state=42
)

# Define data
print("\nTraining shapes:")
print("X_train shape:", x_train.shape)
print("y_department_train shape:", y_dept_train.shape)
print("y_attrition_train shape:", y_attr_train.shape)


Training shapes:
X_train shape: (1176, 15)
y_department_train shape: (1176, 2)
y_attrition_train shape: (1176, 3)


## Create, Compile, and Train the Model

In [157]:
# Build the model
input_dim = x_train.shape[1]
inputs = layers.Input(shape=(input_dim,))

# Shared layers
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.3)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Department branch and hidden layers
dept_branch = layers.Dense(16, activation='relu')(x)
dept_output = layers.Dense(3, activation='softmax', name='department')(dept_branch)

# Attrition branch and hidden layers
attr_branch = layers.Dense(16, activation='relu')(x)
attr_output = layers.Dense(2, activation='softmax', name='attrition')(attr_branch)



In [158]:
# Create the model
model = tf.keras.Model(inputs=inputs, outputs=[dept_output, attr_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department': 'categorical_crossentropy',
        'attrition': 'categorical_crossentropy'  # Use categorical_crossentropy
    },
    metrics={
        'department': 'accuracy',
        'attrition': 'accuracy'
    }
)

# Summarize the model
print("\nModel Summary:")
model.summary()


Model Summary:


Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_19            │ (None, 15)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_76 (Dense)          │ (None, 64)             │          1,024 │ input_layer_19[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_38 (Dropout)      │ (None, 64)             │              0 │ dense_76[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_77 (Dense)          │ (None, 32)             │          2,080 │ dropout_38[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_39 (Dropout)      │ (None, 32)             │              0 │ dense_77[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_78 (Dense)          │ (None, 16)             │            528 │ dropout_39[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_79 (Dense)          │ (None, 16)             │            528 │ dropout_39[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department (Dense)        │ (None, 3)              │             51 │ dense_78[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition (Dense)         │ (None, 2)              │             34 │ dense_79[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,245 (16.58 KB)

 Trainable params: 4,245 (16.58 KB)

 Non-trainable params: 0 (0.00 B)

In [159]:
# Train the model
history = model.fit(
    x_train,
    {
        'department': y_dept_train,
        'attrition': y_attr_train
    },
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - attrition_accuracy: 0.1576 - attrition_loss: 0.6054 - department_accuracy: 0.6230 - department_loss: 1.2613 - loss: 1.8671 - val_attrition_accuracy: 0.0424 - val_attrition_loss: 0.4853 - val_department_accuracy: 0.1992 - val_department_loss: 0.9518 - val_loss: 1.4582
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_accuracy: 0.0431 - attrition_loss: 0.4510 - department_accuracy: 0.1541 - department_loss: 0.9289 - loss: 1.3804 - val_attrition_accuracy: 0.0424 - val_attrition_loss: 0.4892 - val_department_accuracy: 0.2034 - val_department_loss: 0.8269 - val_loss: 1.3436
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_accuracy: 0.0386 - attrition_loss: 0.4896 - department_accuracy: 0.1759 - department_loss: 0.8051 - loss: 1.2949 - val_attrition_accuracy: 0.0424 - val_attrition_loss: 0.4734 - val_department_accuracy: 0.2034 - val_department_loss: 0.7757 - val_loss: 1.2764
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0

In [160]:
# Evaluate the model with the testing data
# Evaluate the model
test_results = model.evaluate(
    x_test,
    {
        'department': y_dept_test,
        'attrition': y_attr_test
    },
    verbose=0
)

In [161]:
# Print the accuracy for both department and attrition
print("\nTest Results:")
print(f"Department Accuracy: {test_results[3]:.4f}")
print(f"Attrition Accuracy: {test_results[4]:.4f}")



Test Results:
Department Accuracy: 0.0408
Attrition Accuracy: 0.1054


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not the best metric. Since the data is imbalanced it can be misleading.
2. I chose Softmax. It is a good choice for multi-class problems. It allows the model to predict the class with the best probability.
3. Using pre-trained models, applying hyperparameter tuning,incrwasing the number of neurons, and adding more layers.